In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(suppress=True)   
pd.set_option('display.float_format', lambda x: '%.5f' % x) 
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)
import seaborn as sns  
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor
from matplotlib.ticker import FuncFormatter
from matplotlib.pyplot import MultipleLocator
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
import joblib
import scipy.stats
from time import *
from tqdm import tnrange, tqdm_notebook
from time import sleep
from matplotlib.mathtext import _mathtext as mathtext
from sklearn.utils import resample
from sklearn.ensemble import VotingRegressor
mathtext.FontConstantsBase.sup1 = 0.3 

In [2]:
var = locals()

In [ ]:
nh3 = pd.read_excel('./df_training.xlsx')

In [ ]:
nh3['Number of fertilizer applications'] = [1 if x >= 2 else 0 for x in nh3['Number of fertilizer applications']]

In [ ]:
stratify_columns = ['Fertilizer type', 'Nitrogen placement', 'Number of fertilizer applications', 'Soil tillage practices', 'Crop type']
bootstrapped_datasets = []

for i in range(5,104):
    var['nh3_' + str(i)] = resample(nh3, stratify=nh3[stratify_columns], replace=True,random_state = i)
    bootstrapped_datasets.append(var['nh3_' + str(i)])
    #print(var['nh3_' + str(i)]['EF'].mean())
    #print(var['nh3_' + str(i)]['Number of fertilizer applications'].sum())
    #print(len(var['nh3_' + str(i)][var['nh3_' + str(i)]['Crop type'] == 'Rice']))
bootstrapped_datasets.append(nh3)

In [ ]:
len(bootstrapped_datasets)

In [ ]:
nh3_orig = nh3.copy()

In [ ]:
for nh3df in bootstrapped_datasets:    
    for i in range(1, len(nh3df['Fertilizer type'].unique()) + 1) :
        nh3df[nh3df['Fertilizer type'].unique()[i - 1]] = np.array(nh3df['Fertilizer type'] == nh3df['Fertilizer type'].unique()[i - 1]).astype(np.int32)
        i += 1
    del nh3df['Fertilizer type']

In [ ]:
for nh3df in bootstrapped_datasets:   
    for i in range(1, len(nh3df['Nitrogen placement'].unique()) + 1) :
        nh3df[nh3df['Nitrogen placement'].unique()[i - 1]] = np.array(nh3df['Nitrogen placement'] == nh3df['Nitrogen placement'].unique()[i - 1]).astype(np.int32)
        i += 1
    del nh3df['Nitrogen placement']

In [ ]:
for nh3df in bootstrapped_datasets:   
    nh3df['Soil tillage practices'] = [1 if x=='NT' else 0 for x in nh3df['Soil tillage practices']]

In [ ]:
for nh3df in bootstrapped_datasets:   
    for i in range(1, len(nh3df['Crop type'].unique()) + 1) :
        nh3df[nh3df['Crop type'].unique()[i - 1]] = np.array(nh3df['Crop type'] == nh3df['Crop type'].unique()[i - 1]).astype(np.int32)
        i += 1
    del nh3df['Crop type']

In [ ]:
for nh3df in bootstrapped_datasets[:-1]: 
    nh3df = sklearn.utils.shuffle(nh3df,random_state=2022)  
    nh3df.index = range(0,len(nh3df))
    nh3df

In [ ]:
nh3 = sklearn.utils.shuffle(nh3,random_state=2022) 
nh3.index = range(0,len(nh3))

In [ ]:
EFraw = nh3['EF'].copy()
#nh3['EF'] = np.log(nh3['EF'])
pt = preprocessing.PowerTransformer(method='box-cox', standardize=False)
nh3['EF'] = pt.fit_transform(nh3['EF'].ravel().reshape(-1, 1))

In [ ]:
for nh3df in bootstrapped_datasets[:-1]: 
    nh3df['EF'] = pt.transform(nh3df['EF'].ravel().reshape(-1, 1))

In [ ]:
X_col_ori = list(nh3.columns)

In [ ]:
X_col_ori.remove('EF')
X_col_ori.remove('region')
X_col_ori.remove('Longitude')
X_col_ori.remove('Latitude')

In [ ]:
X = nh3[X_col_ori]

In [ ]:
for i in range(5,104):
    var['X_'+str(i)] = var['nh3_' + str(i)][X_col_ori]
    var['X_'+str(i)] = var['X_'+str(i)].rename(columns={'Number of fertilizer applications':'FAT',
                      'Soil tillage practices':'STP',
                      'Average daily air temperature':'Tem',
                      'SOC(g/kg)':'SOC','TN(g/kg)':'TN','BD(g/cm3)':'BD','clay(%)':'Clay','CEC(cmol/kg)':'CEC',
                      'Nitrogen application rate':'Nrate',
                      'U':'UOA','P+I':'Water'})

In [ ]:
X = X.rename(columns={'Number of fertilizer applications':'FAT',
                      'Soil tillage practices':'STP',
                      'Average daily air temperature':'Tem',
                      'SOC(g/kg)':'SOC','TN(g/kg)':'TN','BD(g/cm3)':'BD','clay(%)':'Clay','CEC(cmol/kg)':'CEC',
                      'Nitrogen application rate':'Nrate',
                      'U':'UOA','P+I':'Water'})

In [ ]:
X_col = list(X.columns)

In [ ]:
Y = nh3['EF'].values.reshape(len(nh3),1)

In [ ]:
for i in range(5,104):
    var['Y_'+str(i)] = var['nh3_' + str(i)]['EF'].values.reshape(len(var['nh3_' + str(i)]),1)

In [ ]:
X.shape, Y.shape

In [ ]:
var['X_'+str(i)].shape, var['Y_'+str(i)].shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2) 

In [ ]:
x_train.shape, x_test.shape

In [ ]:
y_train.shape, y_test.shape

In [ ]:
n_estimators = range(100,101)
random_state = range(2023,2024)
max_features= range(1,22)
min_samples_leaf = range(1,5)
max_depth = [3, 5, 8, 10, 15, 20, 25, 30, 35, 40, None] 

In [ ]:
param_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
             'random_state': random_state,
             'min_samples_leaf': min_samples_leaf,
             'max_depth': max_depth}

In [ ]:
grid = GridSearchCV(RandomForestRegressor(), param_grid, cv=10,n_jobs = -1,return_train_score=False)

In [ ]:
begin_time = time()

In [ ]:
grid.fit(x_train, y_train.ravel())
#grid.fit(X, Y.ravel())

In [ ]:
end_time = time()
run_time = end_time-begin_time
print ('time cost：',run_time/60, 'min') 

In [ ]:
grid.best_params_

In [ ]:
n_estimators = range(500,501)
random_state = range(2023,2024)
max_features= range(6,22)
min_samples_leaf = range(1,2)
max_depth = [15, 20, 25, 30, 35, 40, None]

In [ ]:
param_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
             'random_state': random_state,
             'min_samples_leaf': min_samples_leaf,
             'max_depth': max_depth}

In [ ]:
grid = GridSearchCV(RandomForestRegressor(), param_grid, cv=10,n_jobs = 7,return_train_score=False)

In [ ]:
begin_time = time()

In [ ]:
grid.fit(x_train, y_train.ravel())
#grid.fit(X, Y.ravel())

In [ ]:
end_time = time()
run_time = end_time-begin_time
print ('time cost：',run_time/60, 'min') 

In [ ]:
grid.best_params_

In [ ]:
grid.score(x_test, y_test)

In [ ]:
grid.score(x_train, y_train)

In [ ]:
np.sqrt(mean_squared_error(grid.predict(x_test), y_test))

In [ ]:
np.sqrt(mean_squared_error(grid.predict(X), Y))

In [ ]:
pd.DataFrame(grid.cv_results_).iloc[grid.best_index_,:]

In [ ]:
RF = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'], max_features=grid.best_params_['max_features'],min_samples_leaf = grid.best_params_['min_samples_leaf'],max_depth = grid.best_params_['max_depth'],oob_score = True,random_state=grid.best_params_['random_state'],n_jobs=14)
RF.fit(x_train, y_train.ravel())

In [ ]:
RF

In [ ]:
joblib.dump(RF,'./RF_V6_alldatatraining_trainingset_100bootstrap_1000Nmax.pkl') #save model

In [ ]:
RF.score(x_test, y_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import max_error
from quantile_forest import RandomForestQuantileRegressor

In [ ]:
sdf = []  
for i in tqdm_notebook(range(5,104)):
    RF = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'], max_features=grid.best_params_['max_features'],min_samples_leaf = grid.best_params_['min_samples_leaf'],max_depth = grid.best_params_['max_depth'],oob_score = False,random_state=grid.best_params_['random_state'],n_jobs=14)
    RF.fit(var['X_'+str(i)], var['Y_'+str(i)].ravel())
    joblib.dump(RF,'./RF_V6_alldatatraining_100bootstrap_1000Nmax_'+ str(i) +'.pkl')
    sdf.append(pt.inverse_transform(np.array(RF.predict(x_test).reshape(-1, 1)))[0])
print(np.mean(sdf))
print(np.std(sdf))
print(np.std(sdf)/np.mean(sdf))
print(np.percentile(sdf, 2.5 ))
print(np.percentile(sdf, 97.5 ))

In [ ]:
RF = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'], max_features=grid.best_params_['max_features'],min_samples_leaf = grid.best_params_['min_samples_leaf'],max_depth = grid.best_params_['max_depth'],oob_score = True,random_state=grid.best_params_['random_state'],n_jobs=14)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
scores = cross_val_score(RF, X, Y, cv=10)

In [ ]:
scores

In [ ]:
scores.mean()

In [ ]:
RF.fit(X, Y.ravel())

In [ ]:
##save model
joblib.dump(RF,'./RF_V6_alldatatraining_100bootstrap_1000Nmax.pkl')
joblib.dump(grid,'./RF_grid_V6_alldatatraining_100bootstrap_1000Nmax.pkl')
joblib.dump(pt,'./boxcox_V6_alldatatraining_100bootstrap_1000Nmax.pkl')